In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    def shift(x, w):
        return convolve(x.reshape((8, 8)), mode='constant', weights=w).ravel()

    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

In [3]:
# Load Data
digits = datasets.load_digits()
X = np.asarray(digits.data, 'float32')
X, Y = nudge_dataset(X, [(yi, yi) for yi in digits.target])
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)


In [4]:
# Models we will use
logistic = MultiOutputRegressor(linear_model.LogisticRegression(solver='lbfgs', max_iter=10000,
                                           multi_class='multinomial'))
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

# #############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 20
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000

In [5]:
# Training RBM-Logistic Pipeline
rbm_features_classifier.fit(X_train, Y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -25.39, time = 1.13s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -23.77, time = 1.49s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -22.94, time = 1.36s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -21.91, time = 1.29s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -21.69, time = 1.37s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -21.06, time = 1.35s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -20.89, time = 1.41s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -20.64, time = 1.37s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -20.36, time = 1.44s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -20.09, time = 1.31s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -20.08, time = 1.32s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -19.82, time = 1.31s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -19.64, time = 0.87s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -19.61, time = 0.54s
[BernoulliRBM] Iteration 15, 

Pipeline(memory=None,
     steps=[('rbm', BernoulliRBM(batch_size=10, learning_rate=0.06, n_components=100, n_iter=20,
       random_state=0, verbose=True)), ('logistic', MultiOutputRegressor(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
           n_jobs=1))])

In [7]:
Y_pred = rbm_features_classifier.predict(X_test)


In [11]:
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test[:,0], Y_pred[:,0])))

Logistic regression using RBM features:
             precision    recall  f1-score   support

          0       1.00      0.98      0.99       174
          1       0.81      0.85      0.83       184
          2       0.88      0.92      0.90       166
          3       0.89      0.80      0.84       194
          4       0.96      0.93      0.94       186
          5       0.87      0.83      0.85       181
          6       0.98      0.96      0.97       207
          7       0.88      0.97      0.93       154
          8       0.77      0.81      0.79       182
          9       0.81      0.79      0.80       169

avg / total       0.89      0.88      0.88      1797




In [12]:
# Training the Logistic regression classifier directly on the pixel
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

MultiOutputRegressor(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
           n_jobs=1)

In [13]:
Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test[:,0], Y_pred[:,0])))

Logistic regression using raw pixel features:
             precision    recall  f1-score   support

          0       0.89      0.94      0.91       174
          1       0.57      0.56      0.57       184
          2       0.78      0.85      0.81       166
          3       0.78      0.76      0.77       194
          4       0.81      0.84      0.82       186
          5       0.76      0.73      0.75       181
          6       0.90      0.88      0.89       207
          7       0.83      0.89      0.86       154
          8       0.72      0.57      0.64       182
          9       0.72      0.76      0.74       169

avg / total       0.78      0.78      0.78      1797


